## Frequencies plotter

In this notebook we load the stored data of frequencies between teams and plot them. The basic structure of the code is to read and then to plot.

First, we import the required libraries.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd

Read the stored NetCDF4 file, where the data is stored.

In [ ]:
# Llegim les dades
dataarray = xr.open_dataset('teammates.nc', engine='scipy')
dataarray

Store the list of all the players.

In [ ]:
# List of all player names (ordered)
players_names = dataarray.player.values

Define a function that will plot the heatmap of each parameter.

In [ ]:
# Function to plot the heatmaps
def heatmap_frequencies(matrix, ax, cmap = 'YlGn'):
    # Plot heatmap
    if cmap == 'Reds':
        ax.imshow(matrix, cmap = cmap, aspect = 'equal', vmin = 0, vmax = 1)
    else:
        ax.imshow(matrix, cmap = cmap, aspect = 'equal')
    
    # Show the values at each cell
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            value = matrix[i,j] # value at that cell

            if np.isnan(value):
                continue
            else:
                if isinstance(value, float): # if values are float
                    ax.text(j, i, "{:.2f}".format(value), ha='center', va='center', fontsize=7, color="black")
                else: # if values are other (int)
                    ax.text(j, i, "{:d}".format(value), ha='center', va='center', fontsize=7, color="black")
        

Plotting. First, select the parameters to plot, and store their corresponding numpy arrays (matrices). We filter the 0 values in the ratio between wins and played games for plotting purposes. Then, initialize the figure and plot each colormap using the previouly-defined function.

In [ ]:
teammates = dataarray['Teammates'].values.astype(int) # number of games played with each teammate
teammates_wins = dataarray['TeammatesWins'].values.astype(int) # number of games won with each teammate
teammatesplayed = dataarray['TeammatesPlayed'].values.astype(float) # percentage of games played with each mate wrt the total games played by player
teammatesplayed_wins = dataarray['TeammatesWinsPlayed'].values.astype(float) # percentage of games played with each mate wrt the total games played by player

teammatesplayed_wins[teammatesplayed_wins == 0.] = np.nan

fig, axs = plt.subplots(figsize=(16,16), ncols = 2, nrows = 2)
axs = axs.flatten()

heatmap_frequencies(teammates, axs[0])
heatmap_frequencies(teammates_wins, axs[1])
heatmap_frequencies(teammatesplayed, axs[2])
heatmap_frequencies(teammatesplayed_wins, axs[3], cmap = 'RdYlGn') # la parella més efectiva és la que té aquest valor més alt

# Add tick labels
axs[0].set_title("Games played")
axs[1].set_title("Games won")
axs[2].set_title("Games played with mate / Total games played by player")
axs[3].set_title("Games won with mate / Total games played with mate")

for i in range(4):
    axs[i].set_xticks(ticks=np.arange(len(players_names)))
    axs[i].set_xticklabels(players_names, rotation=90, fontsize=8)
    axs[i].set_yticks(ticks=np.arange(len(players_names)))
    axs[i].set_yticklabels(players_names, rotation=0, fontsize=8)

    axs[i].set_ylabel("Team mate")
    axs[i].set_xlabel("Player")

plt.savefig('results/frequencies.png', dpi=300, bbox_inches='tight')